In [86]:
import pandas as pd
import os
import pymongo
from pymongo import MongoClient
from pymongo import TEXT


XLS_FILE=r'C:\Users\Utilisateur\OneDrive - IDENEA\Agence\Dossiers en cours\ODYCEE - BAYER - PSE - mai 2022\Documents partagés\Echantillonnage liste du personnel pour entretiens V3.xlsx'
DICT_CONNECT={"database_name" : 'bayer' ,"collection_name" : 'person',"server" : 'localhost',"mongodb_port" : 27018}

def filter_mongoDB(condition,field,database_name = 'mydatabasename' ,collection_name='mycollectionname',mongodb_port = 27017,server='localhost'):
    
    client = MongoClient(server,int(mongodb_port))
    db = client[database_name]
    collection = db[collection_name]

    #result=collection.find(condition,field)
    result_df=pd.DataFrame(list(collection.find(condition,field)))

    return result_df

    
    




def write_df_to_mongoDB(  my_df,\
                          database_name = 'mydatabasename' ,\
                          collection_name = 'mycollectionname',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 100):
    #"""
    #This function take a list and create a collection in MongoDB (you should
    #provide the database name, collection, port to connect to the remoete database,
    #server of the remote database, local port to tunnel to the other machine)
    #
    #---------------------------------------------------------------------------
    #Parameters / Input
    #    my_list: the list to send to MongoDB
    #    database_name:  database name
    #
    #    collection_name: collection name (to create)
    #    server: the server of where the MongoDB database is hosted
    #        Example: server = 'XXX.XXX.XX.XX'
    #    this_machine_port: local machine port.
    #        For example: this_machine_port = '27017'
    #    remote_port: the port where the database is operating
    #        For example: remote_port = '27017'
    #    chunk_size: The number of items of the list that will be send at the
    #        some time to the database. Default is 100.
    #
    #Output
    #    When finished will print "Done"
    #----------------------------------------------------------------------------
    #FUTURE modifications.
    #1. Write to SQL
    #2. Write to csv
    #----------------------------------------------------------------------------
    #30/11/2017: Rafael Valero-Fernandez. Documentation
    #"""



    #To connect
    # import os
    # import pandas as pd
    # import pymongo
    # from pymongo import MongoClient

    client = MongoClient(server,int(mongodb_port))
    db = client[database_name]
    collection = db[collection_name]
    # To write
    collection.delete_many({})  # Destroy the collection
    #aux_df=aux_df.drop_duplicates(subset=None, keep='last') # To avoid repetitions
    my_list = my_df.to_dict('records')


    # Inser chunks of the dataframe
    list_chunked = [my_list[i:i + chunk_size] for i in range(0, len(my_list), chunk_size)]

    for chunk in list_chunked:
        #print ("chunk :",chunk)
        collection.insert_many(chunk) # fill de collection


    print('Done')
    return


def text_search_mongoDB(index,text_search,database_name = 'mydatabasename' ,collection_name = 'mycollectionname',server = 'localhost',mongodb_port = 27017):
    # set connection 
    client = MongoClient(server,int(mongodb_port))
    db = client[database_name]
    collection = db[collection_name]

    # set index 
    #collection.create_index([("prenom",TEXT)], default_language='french')
    #collection.create_index([("nom",pymongo.TEXT),("prenom",pymongo.TEXT),("Email",pymongo.TEXT)])

    for index in collection.list_indexes():
        print(index)
    collection.create_index([("$**",pymongo.TEXT)],name='search_index', default_language='french')
    #condition={ $text: { $search: text_search }}

    results=collection.find({"$text": { "$search": "marketing" }})


    result_df=pd.DataFrame(list(results))

    return result_df




def insert_datas_from_excel(xls_file,database_name = 'mydatabasename' ,collection_name = 'mycollectionname',server = 'localhost',mongodb_port = 27017):
    df=pd.read_excel(xls_file)
    #display(df)
    write_df_to_mongoDB(df,database_name=database_name, collection_name=collection_name, server=server, mongodb_port=mongodb_port)

def get_field_list(database_name = 'mydatabasename' ,collection_name = 'mycollectionname',server = 'localhost',mongodb_port = 27017):
    client = MongoClient(server,int(mongodb_port))
    db = client[database_name]    
    cursor = db[collection_name].find({})
    field_list=[]

    first_row = next(cursor, None)
    if first_row:
       field_list=list(first_row.keys())

    for document in cursor: 
        field_list=list(set(field_list)& set(list(document.keys())))
    print(field_list)
          # print all fields of this document. 





def main():

    insert_datas_from_excel(XLS_FILE,**DICT_CONNECT)

    get_field_list(**DICT_CONNECT)

    results= filter_mongoDB({"service":"Ventes Mktg PH"},{},**DICT_CONNECT)

    results=text_search_mongoDB({ "nom": "text" },'',**DICT_CONNECT)
    results.set_index('nom',inplace=True)
    display(results)



 
if __name__ == '__main__':
    main()
    
    

Done
['VG', 'UnitStru', 'Poste', 'prenom', 'A', 'Sexe', 'Select', 'Unnamed: 24', 'R', 'STANNEUX', 'Domaine', 'Unité structurelle', 'PCS', 'Age du salarié', 'Manager ID', 'STANNEUX.1', 'Ancienneté en année', 'Entretien ALB', 'Désignation entité organisatio', 'Mat.', 'Contrat', 'ManagName', 'SDENIS', 'Email', 'Code PCS : Nomenclature des pr', 'Statut', 'Fait/prévu', 'Poste.1', 'service', '_id', 'nom']
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_fts', 'text'), ('_ftsx', 1)])), ('name', 'search_index'), ('weights', SON([('$**', 1)])), ('default_language', 'french'), ('language_override', 'language'), ('textIndexVersion', 3)])


,_id,service,Select,A,R,VG,Entretien ALB,SDENIS,Fait/prévu,STANNEUX,...,UnitStru,Unité structurelle,Poste,Poste.1,STANNEUX.1,Unnamed: 24,Email,prenom,Manager ID,ManagName
nom,,,,,,,,,,,,,,,,,,,,,
PEROY,62d4584e5de2d2e433c99748,Marketing PH,NaN,X,X,NaN,NaN,NaN,NaN,NaN,...,1901774,Marketing Ophtalmo,1900551,Responsable Marketing,NaN,NaN,MARION.PEROY@BAYER.COM,MARION,NaN,NaN
NDIAYE,62d4584e5de2d2e433c99732,Marketing PH,X,X,X,NaN,NaN,x,NaN,NaN,...,72036729,Marketing Xarelto,72274266,Responsable Marketing,NaN,NaN,SOPHIE.NDIAYE@BAYER.COM,SOPHIE,NaN,NaN
BERGER,62d4584d5de2d2e433c995c1,Marketing PH,NaN,X,X,NaN,NaN,NaN,NaN,NaN,...,1906659,Marketing Oncologie,1901772,Responsable Marketing,NaN,NaN,PAULINE.BERGER@BAYER.COM,PAULINE,NaN,NaN
LELUC,62d4584e5de2d2e433c996de,Marketing PH,X,X,X,NaN,NaN,NaN,NaN,x,...,72084429,Marketing Cardio-Renal,72527937,Responsable Marketing,NaN,NaN,CAROLE.LELUC@BAYER.COM,CAROLE,NaN,NaN
LAMOTHE,62d4584e5de2d2e433c996cc,Marketing PH,NaN,X,X,x,NaN,x,NaN,NaN,...,1906668,Hematologie Ventes Marketing,72417331,Responsable Marketing et Ventes,NaN,NaN,DANIEL.LAMOTHE@BAYER.COM,DANIEL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BENIT,62d4584d5de2d2e433c995bb,Marketing Operations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72060727,Etudes et analyses Pharma département,1906751,Analyste Marché et Ventes Spécialiste,NaN,NaN,STEPHANIE.BENIT@BAYER.COM,STEPHANIE,NaN,NaN
AGNANI-PETIT,62d4584d5de2d2e433c99596,Marketing Operations,X,NaN,NaN,NaN,NaN,x,NaN,NaN,...,72101316,Science des Données & Connaissance clts,72630981,Analyste Expérience Client,NaN,NaN,LAETITIA.AGNANI-PETIT@BAYER.COM,LAETITIA,NaN,NaN
ABOULEHADI,62d4584d5de2d2e433c99590,Marketing Operations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72101316,Science des Données & Connaissance clts,72182242,Expert Gestion de Données et Visualisati,NaN,NaN,MAROUANE.ABOULEHADI@BAYER.COM,MAROUANE,NaN,NaN
